# Real Estate Project Summary Notebook

## Problem Overview

- **Goal**: Predict rental prices in Victoria using internal and external factors

- **Outcome**: Identify areas with the greatest potential for growth, the most liveable and most affordable suburbs and find which internal and external factors have the greatest effect on rental prices

### Purpose

To help rental property investors:

- Identify which features help increase a property’s rental price
- Predict future rental price increases for better investment decisions.
- Identify suburbs with high growth potential for long-term profitability.

## Data Collection

- web scraping
- dataset collection

## Preprocessing

- process raw data
- clean missing values
- merge datasets

## Feature Importance

## Forecasting

## Affordability and Liveability

### Affordability

- It is generally accepted that if a household's rent payment exceeds 30% of that household's income then that house is in housing stress

-  This indicates that housing is unaffordable, with rent consuming a disproportionately large amount of a household's income

- We relied on this principle to create an overall Rental Affordability Index (RAI).

The Rental Affordability Index (RAI) is calculated as:

$$ RAI = \left( \frac{\text{Median Income}}{\text{Qualifying Income}} \right) \times 100 $$

Where:

$$ \text{Qualifying Income} = \frac{\text{Median Rent}}{0.30} $$

#### Calculating RAI for various household typed across all suburbs 

- weekly income amounts for various low-moderate income households were assumed
- all household types affordable weekly rent except student sharehouses were taken from the Homes Victoria Quarterly Rental Report Mar 2024: (https://www.dffh.vic.gov.au/homes-victoria-rental-report-march-quarter-2024-word)
- student sharehouse household types affordable weekly rent was calculated as 30% of income (full amount of Centrelink Youth Allowance with no additional income)  + Centrelink Rental Assistance: (https://www.servicesaustralia.gov.au/sites/default/files/2024-09/co029-2409.pdf)
- median rental data by suburb in Victoria was taken from (https://www.dffh.vic.gov.au/moving-annual-rent-suburb-march-quarter-2024-excel)
- this data included median rents by suburb for each of the different household types i.e. 1 bedroom flat, 2 bedroom flat, 1 bedroom house, 2 bedroom house and 3 bedroom house.
- household income types were then matched up with the median rents of suitably sized bedrooms
- for example, a 1 bedroom flat was matched up with the weekly income of a single on a Jobseeker government payment
- these were then brought together and averaged for each suburb
- the results were mapped and can be seen below:

In [12]:
from IPython.display import IFrame

# Embed the HTML file of the folium map into the notebook
IFrame('../plots/affordability_map.html', width=700, height=500)

#### Limitations and Assumptions

- median rental data was limited and not available for several suburbs in Victoria with quite a few of these suburbs being located in Regional Victoria
- we also assumed weekly income amounts for various low-moderate income households based on Centrelink payments and the most recent quarterly rental report done by Homes Victoria www.dffh.vic.gov.au/homes-victoria-rental-report-march-quarter-2024-word

## Predictions

- analyse results
- suburb forecasts

## Privacy and Security

To ensure we worked securely and ethically, we adopted:
- Sleep timers on web-scrapers
- Adhered to API service quotas
- Private cloud services for storing API keys and datasets